In [1]:
!pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.0/216.0 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [2]:
# for google colab
from google.colab import drive
# mount your Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# for google colab
# copy all files from "HW5" directory in Google drive to current directory
!cp -r ./gdrive/MyDrive/Final/* .

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import csv
import cv2
import random
import os

import keras
from scipy.io import loadmat
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import math
import os
from keras.layers import *
from keras.models import *
from keras.optimizers import *
import numpy as np

#device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
from sklearn.model_selection import *
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.inspection import DecisionBoundaryDisplay

# Data Loading

In [6]:
'''
train = pd.read_csv('train.csv', index_col = 'id') 
test = pd.read_csv('test.csv', index_col = 'id')
submission = pd.read_csv('sample_submission.csv')

# Remove target column
target = train['failure'].copy()
gc.collect()

train.head()
'''
MANIFEST_DIR = "train.csv"
TEST_MANIFEST_DIR = "test.csv"
Batch_size = 500
Long = 26570
Lens = 21256

In [7]:
from sklearn.decomposition import PCA
def make_preprocessor(dataset):
    num_cols = [col for col in dataset if dataset[col].dtype not in ['object']] # numeric type
    cat_cols = [col for col in dataset if dataset[col].dtype in ['object']] # category type
    num_pipe = Pipeline([
      ('impute', SimpleImputer()),
      ('scale', MinMaxScaler()),
    ])
    cat_pipe = Pipeline([
      ('encode', OrdinalEncoder())
    ])
    preprocessor = ColumnTransformer([
      ('numeric', num_pipe, num_cols),
      ('categorical', cat_pipe, cat_cols),
    ])
    return preprocessor

In [8]:
def xs_gen(path=MANIFEST_DIR,batch_size = Batch_size,train=True,Lens=Lens):

    #img_list = pd.read_csv(path)
    if train:
        #img_list = np.array(img_list)[:Lens]
        img_list = X[:Lens]
        print("Found %s train items."%len(img_list))
        #print("list 1 is",img_list[0,-1])
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch
    else:
        #img_list = np.array(img_list)[Lens:]
        img_list = X[Lens:]
        print("Found %s test items."%len(img_list))
        #print("list 1 is",img_list[0,-1])
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch
    while True:
      for i in range(steps):
        batch_list = img_list[i * batch_size : i * batch_size + batch_size]
        np.random.shuffle(batch_list)
        batch_x = np.array([file for file in batch_list[:,0:-1]])
        batch_y = np.array([label for label in batch_list[:,-1]])
        #batch_y = np.array([convert2oneHot(label, 2) for label in batch_list[:,-1]])

        yield batch_x, batch_y


In [9]:
def ts_gen(path=TEST_MANIFEST_DIR,batch_size = Batch_size):

    #img_list = pd.read_csv(path)

    img_list = X_test[:Lens]
    print("Found %s train items."%len(img_list))
    print("list 1 is",img_list[0,-1])
    steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch
    while True:
      for i in range(steps):

        batch_list = img_list[i * batch_size : i * batch_size + batch_size]
        #np.random.shuffle(batch_list)
        batch_x = np.array([file for file in batch_list[:,0:]])
        #batch_y = np.array([convert2oneHot(label,10) for label in batch_list[:,-1]])

        yield batch_x


In [10]:
def convert2oneHot(index, Lens):
    hot = np.zeros((Lens, ))
    hot[int(index)] = 1
    return(hot)

In [11]:
df = pd.read_csv('train.csv')
df = df.drop('id', axis=1)
df = df.drop('product_code', axis=1)
y = df['failure']
X = df.drop('failure', axis=1)
prep = make_preprocessor(X)
X = prep.fit_transform(X)
y = np.array(y).reshape(len(y), 1)
X = np.append(X, y, axis=1)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Build Model

In [12]:
TIME_PERIODS = 24
def build_model(input_shape=(TIME_PERIODS,),num_classes=1):
  model = Sequential()
  model.add(Reshape((TIME_PERIODS, 1), input_shape=input_shape))
  model.add(Conv1D(filters=32, kernel_size=3, activation='relu',input_shape=(TIME_PERIODS,1)))

  model.add(Conv1D(filters=32, kernel_size=3, activation='relu',padding="same"))
  model.add(MaxPooling1D(2, padding='same'))

  model.add(Conv1D(64, 4,strides=2, activation='relu',padding="same"))
  model.add(Conv1D(64, 4,strides=2, activation='relu',padding="same"))
  model.add(MaxPooling1D(2, padding='same'))
  
  model.add(Conv1D(256, 4,strides=2, activation='relu',padding="same"))
  model.add(Conv1D(256, 4,strides=2, activation='relu',padding="same"))
  model.add(MaxPooling1D(2, padding='same'))
  model.add(Conv1D(512, 2,strides=1, activation='relu',padding="same"))
  model.add(Conv1D(512, 2,strides=1, activation='relu',padding="same"))
  model.add(MaxPooling1D(2, padding='same'))
  
  """model.add(Flatten())
  model.add(Dropout(0.3))
  model.add(Dense(256, activation='relu'))"""
  model.add(GlobalAveragePooling1D())
  model.add(Dropout(0.3))
  model.add(Dense(num_classes, activation='sigmoid'))
  
  return(model)


# Training

In [13]:
train_iter = xs_gen()
val_iter = xs_gen(train=False)

ckpt = keras.callbacks.ModelCheckpoint(
    filepath='best_model.{epoch:02d}-{val_loss:.4f}.h5',
    monitor='val_loss', save_best_only=True,verbose=1)

model = build_model()
#model = load_model("best_model.10-0.5101.h5")
opt = Adam(0.0002)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())

model.fit_generator(
    generator=train_iter,
    steps_per_epoch= Lens//Batch_size,
    epochs=20,
    initial_epoch=0,
    validation_data = val_iter,
    validation_steps = (Long - Lens)//Batch_size,
    callbacks=[ckpt],
    )
model.save("finishModel.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 9, 16)             144       
                                                                 
 conv1d_1 (Conv1D)           (None, 5, 16)             2064      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 3, 16)            0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 2, 64)             4160      
                                                                 
 conv1d_3 (Conv1D)           (None, 1, 64)             16448     
                                                        

<ipython-input-13-d3aaa9b00476>:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.7806Found 5314 test items.

Epoch 1: val_loss improved from inf to 0.00000, saving model to best_model.01-0.0000.h5
42/42 [==============================] - 56s 1s/step - loss: 0.0000e+00 - accuracy: 0.7806 - val_loss: 0.0000e+00 - val_accuracy: 0.7908
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.7858
Epoch 2: val_loss did not improve from 0.00000
42/42 [==============================] - 45s 1s/step - loss: 0.0000e+00 - accuracy: 0.7858 - val_loss: 0.0000e+00 - val_accuracy: 0.7908
Epoch 3/20
 4/42 [=>............................] - ETA: 37s - loss: 0.0000e+00 - accuracy: 0.7768

KeyboardInterrupt: ignored

# Testing

In [14]:
df = pd.read_csv('test.csv')
id_ = df['id']
df = df.drop('id', axis=1)
#df = df.drop('product_code', axis=1)
#df = df[['product_code', 'loading', 'attribute_0', 'attribute_1', 'attribute_2',
#       'attribute_3', 'measurement_1', 'measurement_5', 'measurement_6',
#       'measurement_7', 'measurement_8', 'measurement_17']]
print(df.shape)
prep = make_preprocessor(df)
X_test = prep.fit_transform(df)
#print(df.shape)
#pred = model.predict_proba(df)[:,1]
#result = pd.DataFrame({'id': id_, 'failure': pred})
#result.to_csv('submission.csv', index=0)

(20775, 24)


In [15]:
test_iter = ts_gen()
#model = load_model("best_model.19-0.5106.h5")
pres = model.predict_generator(generator=test_iter,steps=math.ceil(20775/Batch_size),verbose=1)
print(pres.shape)
#pres.astype('int32')
#img_list = pd.read_csv(TEST_MANIFEST_DIR)
df = pd.DataFrame()
df["id"] = id_
df["failure"] = pres
df.to_csv("submission.csv",index=None)
test_iter = ts_gen()
for x in test_iter:
    x1 = x[0]
    break

<ipython-input-15-11b0641af291>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pres = model.predict_generator(generator=test_iter,steps=math.ceil(20775/Batch_size),verbose=1)


Found 20775 train items.
list 1 is 1.0
42/42 [==============================] - 3s 66ms/step
(20775, 1)
Found 20775 train items.
list 1 is 1.0
